In [1]:
# https://docs.python.org/3.4/installing/index.html
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
import csv
import shutil
import re
import time

In [2]:
import pandas as pd
import numpy as np

## 1. Main page
### Exploring how to extract names of Statutory Boards and url of their respective single pages

In [14]:
site = "https://www.gov.sg/sgdi/statutory-boards"
hdr = {'User-Agent': 'Mozilla/5.0'}
bookpage = requests.get(site)
soup = BeautifulSoup(bookpage.text, "html.parser")
# print(soup.prettify())

In [15]:
soup.ul

<ul class="site-nav">
<li>
<a href="https://m.me/gov.sg" title="Message Us">MESSAGE US</a>
</li>
<li>
<a href="/subscription-page" title="Subscribe Newsletter">SUBSCRIBE</a>
</li>
<li><a href="/contact-info" title="Contact Info">CONTACT</a>
</li>
<li>
<a href="/feedback-form" title="Feedback">FEEDBACK</a>
</li>
<li>
<a href="/sitemap" title="Sitemap">SITEMAP</a>
</li>
<li>
<a href="http://www.ifaq.gov.sg/govsgfaq" title="FAQ">FAQ</a>
</li>
</ul>

In [16]:
soup.find_all('li')

[<li>
 <a href="https://m.me/gov.sg" title="Message Us">MESSAGE US</a>
 </li>, <li>
 <a href="/subscription-page" title="Subscribe Newsletter">SUBSCRIBE</a>
 </li>, <li><a href="/contact-info" title="Contact Info">CONTACT</a>
 </li>, <li>
 <a href="/feedback-form" title="Feedback">FEEDBACK</a>
 </li>, <li>
 <a href="/sitemap" title="Sitemap">SITEMAP</a>
 </li>, <li>
 <a href="http://www.ifaq.gov.sg/govsgfaq" title="FAQ">FAQ</a>
 </li>, <li class="print">
 <a href="javascript:window.print();">Print</a>
 </li>, <li class="smaller-font">
 <a class="rvfs-decrease" href="#">Smaller font</a>
 </li>, <li class="bigger-font">
 <a class="rvfs-increase" href="#">Bigger font</a>
 </li>, <li class="search">
 <div id="mainmenu_0_mobilesearchbar_0_DivCode">
 <div class="site-search">
 <label for="txtSearchkeyMobile" style="display:none;">Search</label>
 <input class="searchinput" id="txtSearchkeyMobile" onclick="return runScriptEvent(event)" onkeydown="if (event.keyCode == 13) document.getElementByI

In [17]:
# trying to find the row containing the first organisation 
# tried with row index 50 as a start
soup.find_all('li')[50]

<li><a href="/sgdi/ministries/mha/statutory-boards/cra">CASINO REGULATORY AUTHORITY OF SINGAPORE (CRA)</a><a class="external" href="http://www.cra.gov.sg" target="_blank"></a></li>

In [18]:
# working backwards - first organisation is at row index 45
soup.find_all('li')[45].text.strip()

'ACCOUNTING AND CORPORATE REGULATORY AUTHORITY (ACRA)'

In [19]:
# getting the url
soup.find_all('li')[45].a['href']

'/sgdi/ministries/mof/statutory-boards/acra'

In [20]:
# getting the last organisation record
soup.find_all('li')[107].text.strip()

'WORKFORCE SINGAPORE (WSG)'

In [21]:
# getting the url
soup.find_all('li')[107].a['href']

'/sgdi/ministries/mom/statutory-boards/wsg'

In [22]:
titles = soup.find_all('li')

In [23]:
for i in range(45,108):
    print(titles[i].text.strip(), titles[i].a['href'])

ACCOUNTING AND CORPORATE REGULATORY AUTHORITY (ACRA) /sgdi/ministries/mof/statutory-boards/acra
AGENCY FOR SCIENCE, TECHNOLOGY AND RESEARCH (A*STAR) /sgdi/ministries/mti/statutory-boards/astar
AGRI-FOOD & VETERINARY AUTHORITY OF SINGAPORE (AVA) /sgdi/ministries/mnd/statutory-boards/ava
BOARD OF ARCHITECTS (BOA) /sgdi/ministries/mnd/statutory-boards/boa
BUILDING AND CONSTRUCTION AUTHORITY (BCA) /sgdi/ministries/mnd/statutory-boards/bca
CASINO REGULATORY AUTHORITY OF SINGAPORE (CRA) /sgdi/ministries/mha/statutory-boards/cra
CENTRAL PROVIDENT FUND BOARD (CPFB) /sgdi/ministries/mom/statutory-boards/cpfb
CIVIL AVIATION AUTHORITY OF SINGAPORE  (CAAS) /sgdi/ministries/mot/statutory-boards/caas
CIVIL SERVICE COLLEGE (CSC) /sgdi/ministries/pmo/statutory-boards/csc
COMPETITION AND CONSUMER COMMISSION OF SINGAPORE (CCCS) /sgdi/ministries/mti/statutory-boards/cccs
COUNCIL FOR ESTATE AGENCIES (CEA) /sgdi/ministries/mnd/statutory-boards/cea
DEFENCE SCIENCE AND TECHNOLOGY AGENCY (DSTA) /sgdi/ministri

In [26]:
title_db = []
url_db = []
for i in range(45,108):
    name = titles[i].text.strip()
    url = titles[i].a['href']
    title_db.append(name)
    url_db.append(url)

In [27]:
title_db

['ACCOUNTING AND CORPORATE REGULATORY AUTHORITY (ACRA)',
 'AGENCY FOR SCIENCE, TECHNOLOGY AND RESEARCH (A*STAR)',
 'AGRI-FOOD & VETERINARY AUTHORITY OF SINGAPORE (AVA)',
 'BOARD OF ARCHITECTS (BOA)',
 'BUILDING AND CONSTRUCTION AUTHORITY (BCA)',
 'CASINO REGULATORY AUTHORITY OF SINGAPORE (CRA)',
 'CENTRAL PROVIDENT FUND BOARD (CPFB)',
 'CIVIL AVIATION AUTHORITY OF SINGAPORE  (CAAS)',
 'CIVIL SERVICE COLLEGE (CSC)',
 'COMPETITION AND CONSUMER COMMISSION OF SINGAPORE (CCCS)',
 'COUNCIL FOR ESTATE AGENCIES (CEA)',
 'DEFENCE SCIENCE AND TECHNOLOGY AGENCY (DSTA)',
 'ECONOMIC DEVELOPMENT BOARD (EDB)',
 'ENERGY MARKET AUTHORITY (EMA)',
 'ENTERPRISE SINGAPORE  (ESG)',
 'GOVERNMENT TECHNOLOGY AGENCY (GOVTECH)',
 'HEALTH PROMOTION BOARD (HPB)',
 'HEALTH SCIENCES AUTHORITY (HSA)',
 'HOTELS LICENSING BOARD (HLB)',
 'HOUSING & DEVELOPMENT BOARD (HDB)',
 'INFO-COMMUNICATIONS MEDIA DEVELOPMENT AUTHORITY  (IMDA)',
 'INLAND REVENUE AUTHORITY OF SINGAPORE (IRAS)',
 'INSTITUTE OF TECHNICAL EDUCATION (ITE

In [28]:
# total number of organisations
len(title_db)

63

In [29]:
url_db[0]

'/sgdi/ministries/mof/statutory-boards/acra'

In [30]:
"https://www.gov.sg"+url_db[0]

'https://www.gov.sg/sgdi/ministries/mof/statutory-boards/acra'

## 2. Single page
### Exploring how to extract specific information i.e. street address, tel number, fax number, website, email of organisation and also downloading logo images automatically

In [3]:
nextsite = "https://www.gov.sg/sgdi/ministries/mot/statutory-boards/caas"
#nextsite
hdr = {'User-Agent': 'Mozilla/5.0'}
onepage = requests.get(nextsite)
onesoup = BeautifulSoup(onepage.text, "html.parser")
# print(onesoup.prettify())

In [12]:
j = onesoup.find('p', class_ = "street-address")
j

<p class="street-address">Terminal 2 - 60 Airport Boulevard Singapore 819643;
Mailing Address:
Singapore Changi Airport
P O Box1
Singapore 918141</p>

In [13]:
j.text.strip()
#j.get_text().strip()

'Terminal 2 - 60 Airport Boulevard Singapore 819643;\r\nMailing Address:\r\nSingapore Changi Airport\r\nP O Box1\r\nSingapore 918141'

In [30]:
onesoup.find('p', class_ = "tel-info").get_text().strip()

'65421122'

In [31]:
onesoup.find_all('p', class_ = "tel-info")[1].get_text().strip()

'65421231'

In [35]:
b = onesoup.find_all('div', class_ = "agency-info")
b

[<div class="agency-info">
 <h2 class="agency-address-title">Address : </h2><address><p class="street-address">Terminal 2 - 60 Airport Boulevard Singapore 819643;
 Mailing Address:
 Singapore Changi Airport
 P O Box1
 Singapore 918141</p><div class="agency-location"><a href="javascript:void(0);" onclick="showMap('819643')">View agency's location map <span class=" glyphicon glyphicon-map-marker"></span></a></div><p class="tel">Tel</p><p class="tel-info">65421122</p><p class="tel">Fax</p><p class="tel-info">65421231</p><p class="website"><a href="http://www.caas.gov.sg">http://www.caas.gov.sg</a></p></address></div>]

In [51]:
onesoup.find('p', class_ = "website").get_text().strip()

'http://www.caas.gov.sg'

In [27]:
onesoup.find_all('a')

[<a href="/" title="Singapore Government">
 <img alt="Singapore Government crest" class="not-dark" src="/sgdi/~/media/gov/common/crest.png"/><img alt="Singapore Government logo" src="/sgdi/~/media/gov/common/logo.jpg"/>
 </a>, <a href="/" title="Singapore Government">
 <img alt="Singapore Government crest" class="not-dark" src="/sgdi/~/media/gov/common/crest-mobile.png"/><img alt="Singapore Government logo" src="/sgdi/~/media/gov/common/logo-mobile.jpg"/>
 </a>, <a href="javascript:window.print();">Print</a>, <a class="rvfs-decrease" href="#">Smaller font</a>, <a class="rvfs-increase" href="#">Bigger font</a>, <a href="javascript:;">
 <span class="glyphicon glyphicon-menu-hamburger"></span>
 </a>, <a href="/">Home</a>, <a href="/news">News</a>, <a href="/factually">Factually</a>, <a data-toggle="dropdown disabled" href="javascript:void(0);">Microsites<span class="car"></span></a>, <a href="/microsites/budget2018">Budget 2018</a>, <a href="/microsites/abrighterfuture">A Brighter Future<

In [42]:
pattern = re.compile("#")
onesoup.find_all('a', href=pattern)

[<a class="rvfs-decrease" href="#">Smaller font</a>,
 <a class="rvfs-increase" href="#">Bigger font</a>,
 <a href="#" title="Facebook"></a>,
 <a href="#" title="Twitter"></a>,
 <a href="#" title="LinkedIn"></a>,
 <a href="#" title="Sharethis"></a>,
 <a href="#" title="Email"></a>,
 <a href="#">qsm@caas.gov.sg</a>,
 <a data-section="section-1" href="#">SHOW</a>,
 <a data-section="section-2" href="#">SHOW</a>,
 <a aria-controls="DepartmentsDivisions" aria-expanded="false" data-toggle="tab" href="#DepartmentsDivisions" id="DepartmentsDivisions-tab" role="tab">Departments/Divisions</a>,
 <a data-section="mobilesection-DepartmentsDivisions" href="#">SHOW</a>]

In [44]:
onesoup.find_all('a', href=pattern)[7].get_text().strip()

'qsm@caas.gov.sg'

In [54]:
onesoup.find('div', class_ = "agency-logo")

<div class="agency-logo"><img alt="CAAS" src="/sgdi/~/media/sgdi/caas.jpg"/></div>

In [58]:
onesoup.find_all('img')

[<img alt="Singapore Government crest" class="not-dark" src="/sgdi/~/media/gov/common/crest.png"/>,
 <img alt="Singapore Government logo" src="/sgdi/~/media/gov/common/logo.jpg"/>,
 <img alt="Singapore Government crest" class="not-dark" src="/sgdi/~/media/gov/common/crest-mobile.png"/>,
 <img alt="Singapore Government logo" src="/sgdi/~/media/gov/common/logo-mobile.jpg"/>,
 <img src="/html/gov/img/ui/ic-officer.png"/>,
 <img alt="CAAS" src="/sgdi/~/media/sgdi/caas.jpg"/>]

In [65]:
logourl = onesoup.find_all('img')[-1]['src']
logourl

'/sgdi/~/media/sgdi/caas.jpg'

In [66]:
"https://www.gov.sg"+logourl

'https://www.gov.sg/sgdi/~/media/sgdi/caas.jpg'

In [76]:
logourl[-4:]

'.jpg'

In [67]:
logoname = onesoup.find_all('img')[-1]['alt']
logoname

'CAAS'

In [77]:
logoname+logourl[-4:]

'CAAS.jpg'

In [78]:
r = requests.get("https://www.gov.sg"+logourl, stream=True, headers={'User-agent': 'Mozilla/5.0'})
if r.status_code == 200:
    with open(logoname+logourl[-4:], 'wb') as f:
        r.raw.decode_content = True
        shutil.copyfileobj(r.raw, f)

## 2.1 Another single page
### Choose any other single organisation page to check if html structure is always consistent  

In [3]:
nextsite = "https://www.gov.sg/sgdi/ministries/mnd/statutory-boards/bca"
#nextsite = url_db[0]
hdr = {'User-Agent': 'Mozilla/5.0'}
page2 = requests.get(nextsite)
soup2 = BeautifulSoup(page2.text, "html.parser")

In [7]:
pattern = re.compile("#")
soup2.find_all('a', href=pattern)

[<a class="rvfs-decrease" href="#">Smaller font</a>,
 <a class="rvfs-increase" href="#">Bigger font</a>,
 <a href="#" title="Facebook"></a>,
 <a href="#" title="Twitter"></a>,
 <a href="#" title="LinkedIn"></a>,
 <a href="#" title="Sharethis"></a>,
 <a href="#" title="Email"></a>,
 <a href="#">bca_enquiry@bca.gov.sg</a>,
 <a data-section="section-1" href="#">SHOW</a>,
 <a data-section="section-2" href="#">SHOW</a>,
 <a aria-controls="DepartmentsDivisions" aria-expanded="false" data-toggle="tab" href="#DepartmentsDivisions" id="DepartmentsDivisions-tab" role="tab">Departments/Divisions</a>,
 <a data-section="mobilesection-DepartmentsDivisions" href="#">SHOW</a>]

In [8]:
soup2.find_all('a', href=pattern)[7].get_text().strip()

'bca_enquiry@bca.gov.sg'

In [9]:
soup2.find_all('img')

[<img alt="Singapore Government crest" class="not-dark" src="/sgdi/~/media/gov/common/crest.png"/>,
 <img alt="Singapore Government logo" src="/sgdi/~/media/gov/common/logo.jpg"/>,
 <img alt="Singapore Government crest" class="not-dark" src="/sgdi/~/media/gov/common/crest-mobile.png"/>,
 <img alt="Singapore Government logo" src="/sgdi/~/media/gov/common/logo-mobile.jpg"/>,
 <img src="/html/gov/img/ui/ic-officer.png"/>,
 <img alt="BCA" src="/sgdi/~/media/sgdi/bca.gif"/>]

In [11]:
logourl = soup2.find_all('img')[-1]['src']
logourl

'/sgdi/~/media/sgdi/bca.gif'

In [12]:
logoname = soup2.find_all('img')[-1]['alt']
logoname

'BCA'

In [13]:
r = requests.get("https://www.gov.sg"+logourl, stream=True, headers={'User-agent': 'Mozilla/5.0'})
if r.status_code == 200:
    with open(logoname+logourl[-4:], 'wb') as f:
        r.raw.decode_content = True
        shutil.copyfileobj(r.raw, f)

## 3. Putting everything together
#### In the event of a disconnection, re-run the code below for sites not yet crawled by updating the range below

In [3]:
site = "https://www.gov.sg/sgdi/statutory-boards"
hdr = {'User-Agent': 'Mozilla/5.0'}
bookpage = requests.get(site)
soup = BeautifulSoup(bookpage.text, "html.parser")
titles = soup.find_all('li')
title_db = []
url_db = []
data = []

for i in range(45,108):
    name = titles[i].text.strip()
    url = titles[i].a['href']
    title_db.append(name)
    url_db.append(url)
    
for i in range(0,len(title_db)): 
#for i in range(0,3): 
    name = title_db[i]
    nextsite = "https://www.gov.sg"+url_db[i]
    hdr = {'User-Agent': 'Mozilla/5.0'}
    onebookpage = requests.get(nextsite)
    onesoup = BeautifulSoup(onebookpage.text, "html.parser")
    #print(onesoup.prettify())
    try:
        address = onesoup.find('p', class_ = "street-address").text.strip()
    except: 
        address = "NA"
    try:
        tel = onesoup.find('p', class_ = "tel-info").get_text().strip()
    except:
        tel = "NA"
    try:
        fax = onesoup.find_all('p', class_ = "tel-info")[1].get_text().strip()
    except:
        fax = "NA"
    try:
        website = onesoup.find('p', class_ = "website").get_text().strip()
    except:
        website = "NA"  
    try:
        pattern = re.compile("#")
        email = onesoup.find_all('a', href=pattern)[7].get_text().strip()
    except: 
        email = "NA"
    data.append((name,address,tel,fax,website,email))
    try:
        logourl = onesoup.find_all('img')[-1]['src']
        logoname = onesoup.find_all('img')[-1]['alt']
        r = requests.get("https://www.gov.sg"+logourl, stream=True, headers={'User-agent': 'Mozilla/5.0'})
        if r.status_code == 200:
            with open(logoname+logourl[-4:], 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
    except:
        pass
    print(i)
    time.sleep(2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62


In [4]:
len(data)

63

In [5]:
df = pd.DataFrame(np.array(data))

In [6]:
df.columns = ['name','address','tel','fax','website','email']

In [7]:
df.head()

,name,address,tel,fax,website,email
0,ACCOUNTING AND CORPORATE REGULATORY AUTHORITY ...,10 Anson Road \r\n#05-01/15 International Plaz...,62486028 (ACRA Helpdesk),NA,http://www.acra.gov.sg,http://www.acra.gov.sg/askacra (Enquiry Websit...
1,"AGENCY FOR SCIENCE, TECHNOLOGY AND RESEARCH (A...",1 Fusionopolis Way\r\n#20-10 Connexis North\r\...,68266111,67771711,http://www.a-star.edu.sg,contact@a-star.edu.sg
2,AGRI-FOOD & VETERINARY AUTHORITY OF SINGAPORE ...,"52, Jurong Gateway Road #14-01\r\nSingapore 60...",68052992 Lines For Public Enquiries/Feedback:,63341831,http://www.ava.gov.sgwww.ava.gov.sg/contactus,www.ava.gov.sg/contactus
3,BOARD OF ARCHITECTS (BOA),"5 Maxwell Road\r\n#01-03, Tower Block, MND Com...",62225295,62224452,http://www.boa.gov.sg,boarch@singnet.com.sg
4,BUILDING AND CONSTRUCTION AUTHORITY (BCA),52 Jurong Gateway Road\r\n#11-01 Singapore 608550,1800-3425222,63344287,http://www.bca.gov.sgbca_enquiry@bca.gov.sg,bca_enquiry@bca.gov.sg


In [8]:
df.to_csv('statboards.csv')

#### Resources: 
http://altitudelabs.com/blog/web-scraping-with-python-and-beautiful-soup/
http://web.stanford.edu/~zlotnick/TextAsData/Web_Scraping_with_Beautiful_Soup.html  
https://www.crummy.com/software/BeautifulSoup/bs4/doc/  
https://www.dataquest.io/blog/web-scraping-beautifulsoup/